# Week 13. Problem set

Evgeny Bobkunov

e.bobkunov@innopolis.university

SD-03

1. Implement a relational program `treeo` that checks whether a given term is a valid tree.

```
(run* (q) (treeo 'empty))
; '(_.0)

(run* (q) (treeo '(node 1 empty (node 2 (node 3 empty empty) empty))))
; '(_.0)

(run 5 (q) (treeo q))
; '(empty
; (node _.0 empty empty)
; (node _.0 empty (node _.1 empty empty))
; (node _.0 (node _.1 empty empty) empty)
; (node _.0 empty (node _.1 empty (node _.2 empty empty))))
```

In [35]:
(require minikanren)
(require minikanren/matche)

(define treeo
  (lambda (t)
    (conde
     [(== t 'empty)]
     [(fresh (value left right)
        (== t `(node ,value ,left ,right))
        (treeo left)
        (treeo right))])))

(run* (q) (treeo 'empty))

'(_.0)

In [37]:
(run* (q) (treeo '(node 1 empty (node 2 (node 3 empty empty) empty))))

'(_.0)

In [39]:
(run 5 (q) (treeo q))

'(empty (node _.0 empty empty) (node _.0 empty (node _.1 empty empty)) (node _.0 (node _.1 empty empty) empty) (node _.0 empty (node _.1 empty (node _.2 empty empty))))

2. Implement predicate `prefix-subtreeo` such that `(prefix-subtreeo small large)` is satisfied when `small` is contained in `large`:

    1. an empty tree is contained in any tree;
    2. a non-empty tree is contained in another non-empty tree when they have the same root and both subtrees are contained in the other subtrees respectively;
  
```
(run* (q)
  (prefix-subtreeo q
    '(node 3
      (node 1 empty
        (node 2 empty empty))
        (node 4 empty empty))))

; '(empty
; (node 3 empty empty)
; (node 3 empty (node 4 empty empty))
; (node 3 (node 1 empty empty) empty)
; (node 3 (node 1 empty empty) (node 4 empty empty))
; (node 3 (node 1 empty (node 2 empty empty)) empty)
; (node 3 (node 1 empty (node 2 empty empty)) (node 4 empty empty)))

(run* (q)
      (prefix-subtreeo
       '(node 3
              (node 1 empty
                    (node 2 empty empty))
              (node 4 empty empty))
        q))

; '((node 3 (node 1 _.0 (node 2 _.1 _.2)) (node 4 _.3 _.4)))
```

In [104]:
(define prefix-subtreeo
  (lambda (small large)
    (conde
      [(== 'empty small)]
      
      [(fresh (small-value small-left small-right
              large-value large-left large-right)
        (== `(node ,small-value ,small-left ,small-right) small)
        (== `(node ,large-value ,large-left ,large-right) large)
        (== small-value large-value)
        (prefix-subtreeo small-left large-left)
        (prefix-subtreeo small-right large-right))])))

In [106]:
(run* (q)
  (prefix-subtreeo q
    '(node 3
      (node 1 empty
        (node 2 empty empty))
        (node 4 empty empty))))

'(empty (node 3 empty empty) (node 3 empty (node 4 empty empty)) (node 3 (node 1 empty empty) empty) (node 3 (node 1 empty empty) (node 4 empty empty)) (node 3 (node 1 empty (node 2 empty empty)) empty) (node 3 (node 1 empty (node 2 empty empty)) (node 4 empty empty)))

In [108]:
(run* (q)
  (prefix-subtreeo
    '(node 3
      (node 1 empty
          (node 2 empty empty))
      (node 4 empty empty))
    q))

'((node 3 (node 1 _.0 (node 2 _.1 _.2)) (node 4 _.3 _.4)))

3. Implement predicate `preordero` such that `(preorder tree vals)` is satisfied when `vals` contains exactly of values from `tree` in **preorder traversal** order.

```
(run* (q) (preordero
  '(node 1 (node 2 (node 3 empty empty) empty) (node 4 empty empty))
  '(3 2 1 4)))
; '()


(run* (q) (preordero
  q
  '(1 2 3)))
; '((node 1 empty (node 2 empty (node 3 empty empty)))
; (node 1 empty (node 2 (node 3 empty empty) empty))
; (node 1 (node 2 empty empty) (node 3 empty empty))
; (node 1 (node 2 empty (node 3 empty empty)) empty)
; (node 1 (node 2 (node 3 empty empty) empty) empty))


(run 7 (q) (fresh (xs) (preordero q xs)))
; '(empty
; (node _.0 empty empty)
; (node _.0 empty (node _.1 empty empty))
; (node _.0 (node _.1 empty empty) empty)
; (node _.0 empty (node _.1 empty (node _.2 empty empty)))
; (node _.0 (node _.1 empty empty) (node _.2 empty empty))
; (node _.0 empty (node _.1 (node _.2 empty empty) empty)))
```

In [11]:
(define appendo
  (lambda (l1 l2 out)
    (conde
      [(== '() l1) (== l2 out)]
      [(fresh (a d res)
         (== `(,a . ,d) l1)
         (== `(,a . ,res) out)
         (appendo d l2 res))])))

(define preordero
  (lambda (tree vals)
    (conde
      [(== 'empty tree) (== '() vals)]
      [(fresh (value left right rest-vals)
         (== `(node ,value ,left ,right) tree)
         (fresh (left-vals right-vals)
           (== `(,value . ,rest-vals) vals)
           (appendo left-vals right-vals rest-vals)
           (preordero left left-vals)
           (preordero right right-vals)))])))


In [13]:
(run* (q)
      (preordero
       '(node 1 (node 2 (node 3 empty empty) empty) (node 4 empty empty))
       '(3 2 1 4)))

'()

In [15]:
(run* (q)
      (preordero q '(1 2 3)))

'((node 1 empty (node 2 empty (node 3 empty empty))) (node 1 empty (node 2 (node 3 empty empty) empty)) (node 1 (node 2 empty empty) (node 3 empty empty)) (node 1 (node 2 empty (node 3 empty empty)) empty) (node 1 (node 2 (node 3 empty empty) empty) empty))

In [17]:
(run 7 (q) 
     (fresh (xs) 
           (preordero q xs)))

'(empty (node _.0 empty empty) (node _.0 empty (node _.1 empty empty)) (node _.0 (node _.1 empty empty) empty) (node _.0 empty (node _.1 empty (node _.2 empty empty))) (node _.0 (node _.1 empty empty) (node _.2 empty empty)) (node _.0 empty (node _.1 (node _.2 empty empty) empty)))

4. Implement predicates `in-ordero` and `post-ordero`, analogous to the previous exercise, but for
**in-order** and **post-order** traversals correspondingly.

```
(run* (q) (post-ordero
  q
  '(1 2 3)))
; '((node 3 empty (node 1 empty (node 2 empty empty)))
; (node 3 empty (node 1 (node 2 empty empty) empty))
; (node 3 (node 1 empty empty) (node 2 empty empty))
; (node 3 (node 1 empty (node 2 empty empty)) empty)
; (node 3 (node 1 (node 2 empty empty) empty) empty))

(run* (q) (in-ordero
  q
  '(1 2 3)))
; '((node 1 empty (node 2 empty (node 3 empty empty)))
; (node 1 empty (node 2 (node 3 empty empty) empty))
; (node 2 (node 1 empty empty) (node 3 empty empty))
; (node 3 (node 1 empty (node 2 empty empty)) empty)
; (node 3 (node 1 (node 2 empty empty) empty) empty))
```

In [53]:
(define in-ordero
  (lambda (tree order)
    (conde
     [(== tree 'empty)
      (== order '())]
     [(fresh (value left right left-order right-order mid-order)
        (== tree `(node ,value ,left ,right))
        (in-ordero left left-order)
        (in-ordero right right-order)
        (appendo left-order (list value) mid-order)
        (appendo mid-order right-order order))])))

(define post-ordero
  (lambda (tree order)
    (conde
     [(== tree 'empty)
      (== order '())]
     [(fresh (value left right left-order right-order mid-order)
        (== tree `(node ,value ,left ,right))
        (post-ordero left left-order)
        (post-ordero right right-order)
        (appendo left-order right-order mid-order)
        (appendo mid-order (list value) order))])))

(define appendo
  (lambda (xs ys zs)
    (conde
     [(== xs '()) (== ys zs)]
     [(fresh (head tail rest)
        (== xs `(,head . ,tail))
        (== zs `(,head . ,rest))
        (appendo tail ys rest))])))


In [55]:
(run 5 (q) (post-ordero
  q
  '(1 2 3)))

'((node 3 (node 1 empty empty) (node 2 empty empty)) (node 3 empty (node 2 empty (node 1 empty empty))) (node 3 empty (node 2 (node 1 empty empty) empty)) (node 3 (node 2 empty (node 1 empty empty)) empty) (node 3 (node 2 (node 1 empty empty) empty) empty))

In [57]:
(run 5 (q) (in-ordero
  q
  '(1 2 3)))

'((node 2 (node 1 empty empty) (node 3 empty empty)) (node 1 empty (node 2 empty (node 3 empty empty))) (node 1 empty (node 3 (node 2 empty empty) empty)) (node 3 (node 1 empty (node 2 empty empty)) empty) (node 3 (node 2 (node 1 empty empty) empty) empty))